In [1]:
import os
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.regularizers import l2
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import keras_tuner as kt

In [2]:
# set random seed
np.random.seed(69)
tf.random.set_seed(69)

In [3]:
# load sequential data
def load_sequential_data(folders):
    """ load csvs as sequences, removing the frame column """
    data, labels = [], []

    for folder, _ in folders:
        if os.path.exists(folder):
            for file in os.listdir(folder):
                if file.endswith('.csv'):
                    file_path = os.path.join(folder, file)
                    df = pd.read_csv(file_path)

                    label = df.iloc[0, 0]  # extract label
                    features = df.iloc[:, 2:].values  # remove frame only keeps movement data
                    
                    data.append(features)
                    labels.append(label)
        else:
            print(f"warning: folder {folder} not found.")

    return np.array(data), np.array(labels)

In [27]:
# define training folders
train_folders = [
    ('../rat_dance_csv/train', 1),
    ('../neg_control_csv/train', 0)
]

# load dataset
X, y = load_sequential_data(train_folders)

# normalize features
scaler = StandardScaler()
# normalize each sequence separately
X = np.array([scaler.fit_transform(sample) for sample in X])

# shuffle dataset
indices = np.random.permutation(len(X))
X, y = X[indices], y[indices]

# reshape X for RNN --samples, timesteps, features--
timesteps, features = X.shape[1], X.shape[2]
X = X.reshape(len(X), timesteps, features)

# use entire training set for tuning 
X_train, y_train = X, y 

# function to build the model for hyperparameter tuning
def build_model(hp):
    model = keras.Sequential([
        keras.layers.LSTM(
            units=hp.Int("units", min_value=32, max_value=128, step=16),
            return_sequences=True,
            recurrent_dropout=hp.Float("recurrent_dropout", 0.1, 0.5, step=0.1),
            input_shape=(timesteps, features),
        ),
        keras.layers.Dropout(hp.Float("dropout", 0.3, 0.7, step=0.1)),
        keras.layers.LSTM(
            units=hp.Int("units_lstm2", min_value=16, max_value=64, step=16),
            return_sequences=False
        ),
        keras.layers.Dropout(hp.Float("dropout", 0.3, 0.7, step=0.1)),
        keras.layers.Dense(
            hp.Int("dense_units", 16, 64, step=16), activation="relu", kernel_regularizer=l2(0.01)
        ),
        keras.layers.Dense(1, activation="sigmoid"),
    ])

    model.compile(
        optimizer=keras.optimizers.Adam(
            learning_rate=hp.Choice("learning_rate", [0.001, 0.0005, 0.0001])
        ),
        loss="binary_crossentropy",
        metrics=["accuracy"],
    )
    return model

# use keras tuner to search for best hyperparameters
tuner = kt.RandomSearch(
    hypermodel=build_model,
    objective="loss", 
    max_trials=15, 
    executions_per_trial=2, 
    directory="delete_me_post_search_too",
    project_name="lstm_tuning",
)

# search for best hyperparameters
tuner.search(X_train, y_train, epochs=10, batch_size=32, verbose=1) 

# get best hyperparameters
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]

# print best values
print(f"Best LSTM units: {best_hps.get('units')}")
print(f"Best dropout: {best_hps.get('dropout')}")
print(f"Best recurrent dropout: {best_hps.get('recurrent_dropout')}")
print(f"Best dense units: {best_hps.get('dense_units')}")
print(f"Best learning rate: {best_hps.get('learning_rate')}")


Trial 15 Complete [00h 00m 22s]
loss: 0.7681911587715149

Best loss So Far: 0.7338444888591766
Total elapsed time: 00h 04m 46s
Best LSTM units: 48
Best dropout: 0.5
Best recurrent dropout: 0.30000000000000004
Best dense units: 16
Best learning rate: 0.001


In [33]:
# create final model using best hyperparameters
final_model = tuner.hypermodel.build(best_hps)

# define early stopping
early_stopping = keras.callbacks.EarlyStopping(
    monitor="val_loss", patience=5, restore_best_weights=True
)

# perform 5-fold cross-validation
cv_accuracies = []

for train_index, val_index in kf.split(X, y):
    X_train, X_val = X[train_index], X[val_index]
    y_train, y_val = y[train_index], y[val_index]

    final_model.fit(
        X_train, y_train, epochs=15, batch_size=32, verbose=0,
        validation_data=(X_val, y_val), callbacks=[early_stopping]
    )

    val_loss, val_acc = final_model.evaluate(X_val, y_val, verbose=0)
    cv_accuracies.append(val_acc)

# print cross validation accuracy
cross_val_acc = np.mean(cv_accuracies)
print(f"cross validation accuracy: {cross_val_acc:.4f}")


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


cross-validation accuracy: 0.7944


In [35]:
# load validation dataset
val_folders = [
    ('../rat_dance_csv/val', 1),
    ('../neg_control_csv/val', 0)
]

X_val, y_val = load_sequential_data(val_folders)
X_val = np.array([scaler.transform(sample) for sample in X_val])  
X_val = X_val.reshape(len(X_val), timesteps, features)  

# evaluate on validation set
y_pred_prob = final_model.predict(X_val)
y_pred = (y_pred_prob > 0.5).astype(int)

accuracy = accuracy_score(y_val, y_pred)
class_report = classification_report(y_val, y_pred, target_names=["negative control (0)", "ratdance (1)"])
conf_matrix = confusion_matrix(y_val, y_pred)

# show results
print(f"\nvalidation performance:")
print(f"accuracy: {accuracy:.4f}")
print("\nclassification report:")
print(class_report)
print("\nconfusion matrix:")
print(conf_matrix)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step

validation performance:
accuracy: 0.5000

classification report:
                      precision    recall  f1-score   support

negative control (0)       0.50      0.29      0.36         7
        ratdance (1)       0.50      0.71      0.59         7

            accuracy                           0.50        14
           macro avg       0.50      0.50      0.48        14
        weighted avg       0.50      0.50      0.48        14


confusion matrix:
[[2 5]
 [2 5]]
